In [ ]:
import pandas as pd
import numpy as np
from keras.utils import np_utils
from skimage.transform import resize
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
data = pd.read_csv(r"/content/drive/MyDrive/dataset/ckextended.csv")


In [ ]:
image_size = (48, 48)
pixels = data['pixels'].tolist()
images = np.array([np.array(pixels[i].split(' ')).astype('float32').reshape(48, 48) for i in range(len(pixels))])
resized_images = np.array([resize(image, image_size, mode='reflect') for image in images])
normalized_images = resized_images / 255.0
labels = data['emotion'].values

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(normalized_images, labels, test_size=0.2, random_state=42)

num_classes = len(np.unique(y_train))
y_train = np_utils.to_categorical(np.array(y_train), num_classes)
y_test = np_utils.to_categorical(np.array(y_test), num_classes)

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam

num_classes= 7

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(48, 48, 1)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.35))

model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.6))

model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.75))
model.add(Dense(num_classes, activation='softmax'))

In [ ]:
print(model.summary())
opt = Adam(learning_rate=0.0001)
model.compile(loss='categorical_crossentropy', optimizer= opt, metrics=['accuracy'])
history= model.fit(X_train.reshape(-1, 48, 48, 1),
                   y_train,
                   batch_size=64,
                   epochs=100,
                   validation_data=(X_test.reshape(-1, 48, 48, 1), y_test))

In [ ]:
score = model.evaluate(X_test.reshape(-1, 48, 48, 1), y_test)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
import matplotlib.pyplot as plt
fig , ax = plt.subplots(1,2)
train_acc = history.history['accuracy']
train_loss = history.history['loss']
fig.set_size_inches(12,4)
ax[0].plot(history.history['accuracy'])
ax[0].plot(history.history['val_accuracy'])
ax[0].set_title('Training Accuracy vs Validation Accuracy')
ax[0].set_ylabel('Accuracy')
ax[0].set_xlabel('Epoch')
ax[0].legend(['Train', 'Validation'], loc='upper left')
ax[1].plot(history.history['loss'])
ax[1].plot(history.history['val_loss'])
ax[1].set_title('Training Loss vs Validation Loss')
ax[1].set_ylabel('Loss')
ax[1].set_xlabel('Epoch')
ax[1].legend(['Train', 'Validation'], loc='upper right')
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

# Make predictions on the test set
y_pred_probs = model.predict(X_test.reshape(-1, 48, 48, 1))
y_pred = np.argmax(y_pred_probs, axis=1)

# Define class labels and emotions
class_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
emotions = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

# Generate the confusion matrix
confusion_mtx = confusion_matrix(np.argmax(y_test, axis=1), y_pred)

# Plotting the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(confusion_mtx, annot=True, fmt="d", cmap="Blues")
plt.title("Confusion Matrix")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.xticks(np.arange(len(class_labels)), class_labels)
plt.yticks(np.arange(len(class_labels)), class_labels)
plt.show()

# Generate the classification report
report = classification_report(np.argmax(y_test, axis=1), y_pred, target_names=emotions)
print(report)